In [17]:
import socket
import spiceypy as spice
import time

In [19]:
spice.furnsh("E:/SPICE/naif0012.tls")
spice.furnsh("E:/SPICE/pck00010.tpc")
spice.furnsh("E:/SPICE/earth_000101_250227_241201.bpc")
spice.furnsh("E:/SPICE/moon_pa_de421_1900-2050.bpc")
spice.furnsh("E:/SPICE/de430.bsp") 

In [21]:
HOST = '127.0.0.1'
PORT = 12345
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(1)

print(f"TCP Sunucusu {HOST}:{PORT} adresinde dinleniyor...")
conn, addr = server_socket.accept()
print(f"Bağlantı sağlandı: {addr}")

TCP Sunucusu 127.0.0.1:12345 adresinde dinleniyor...
Bağlantı sağlandı: ('127.0.0.1', 52666)


In [22]:
start_time = spice.str2et("2000-02-18T18:45:00") 
interval = 0.25 
duration = 1000

current_time = start_time
end_time = start_time + duration
next_send_time = time.perf_counter()

In [23]:
try:
    while current_time <= end_time:
        telemetry = []
        now = time.perf_counter()

        if now >= next_send_time:
            
            state_earth, light_earth = spice.spkgeo(targ=399, et=current_time, ref="ECLIPJ2000", obs=10)
            state_moon, light_moon = spice.spkgeo(targ=301, et=current_time, ref="ECLIPJ2000", obs=10) 
        
            rotation_earth = spice.pxform("IAU_EARTH", "ECLIPJ2000", current_time)
            rotation_moon = spice.pxform("IAU_MOON", "ECLIPJ2000", current_time)
            pos_earth = state_earth[:3]
            pos_moon = state_moon[:3]
                
            roll1, pitch1, yaw1 = spice.m2eul(rotation_earth, 1, 2, 3)
            roll_earth = spice.convrt(roll1, "RADIANS", "DEGREES")
            pitch_earth = spice.convrt(pitch1, "RADIANS", "DEGREES")
            yaw_earth = spice.convrt(yaw1, "RADIANS", "DEGREES")
            rot_earth = [roll_earth,pitch_earth,yaw_earth]
        
            roll2, pitch2, yaw2 = spice.m2eul(rotation_moon, 1, 2, 3)
            roll_moon = spice.convrt(roll2, "RADIANS", "DEGREES")
            pitch_moon = spice.convrt(pitch2, "RADIANS", "DEGREES")
            yaw_moon = spice.convrt(yaw2, "RADIANS", "DEGREES")
            rot_moon = [roll_moon,pitch_moon,yaw_moon]
        
            utc_time = spice.et2utc(current_time, "C", 3)
                
            print(f"{pos_earth[0]:.3f},{pos_earth[1]:.3f},{pos_earth[2]:.3f},{rot_earth[0]:.3f},{rot_earth[1]:.3f},{rot_earth[2]:.3f},{pos_moon[0]:.3f},{pos_moon[1]:.3f},{pos_moon[2]:.3f},{rot_moon[0]:.3f},{rot_moon[1]:.3f},{rot_moon[2]:.3f}")
            telemetry = (f"{pos_earth[0]:.3f},{pos_earth[1]:.3f},{pos_earth[2]:.3f},{rot_earth[0]:.3f},{rot_earth[1]:.3f},{rot_earth[2]:.3f},{pos_moon[0]:.3f},{pos_moon[1]:.3f},{pos_moon[2]:.3f},{rot_moon[0]:.3f},{rot_moon[1]:.3f},{rot_moon[2]:.3f}")
            conn.sendall((telemetry + "\n").encode())
                
            next_send_time += interval
            current_time += interval

except KeyboardInterrupt:
    print("Bağlantı kapatılıyor...")

finally:
    conn.close()
    server_socket.close()
    
    spice.unload("E:/SPICE/naif0012.tls")
    spice.unload("E:/SPICE/pck00010.tpc")
    spice.unload("E:/SPICE/earth_000101_250227_241201.bpc")
    spice.unload("E:/SPICE/moon_pa_de421_1900-2050.bpc")
    spice.unload("E:/SPICE/de430.bsp")

-127297787.340,75201066.335,-60.931,23.439,-0.001,-69.251,-127567062.698,75449389.540,7899.520,-0.797,1.303,45.468
-127297791.246,75201059.898,-60.931,23.439,-0.001,-69.253,-127567066.791,75449382.910,7899.544,-0.797,1.303,45.468
-127297795.153,75201053.460,-60.931,23.439,-0.001,-69.254,-127567070.884,75449376.280,7899.568,-0.797,1.303,45.468
-127297799.060,75201047.023,-60.931,23.439,-0.001,-69.255,-127567074.976,75449369.649,7899.591,-0.797,1.303,45.468
-127297802.967,75201040.585,-60.932,23.439,-0.001,-69.256,-127567079.069,75449363.019,7899.615,-0.797,1.303,45.468
-127297806.874,75201034.148,-60.932,23.439,-0.001,-69.257,-127567083.162,75449356.389,7899.639,-0.797,1.303,45.468
-127297810.781,75201027.710,-60.932,23.439,-0.001,-69.258,-127567087.255,75449349.758,7899.663,-0.797,1.303,45.468
-127297814.687,75201021.273,-60.933,23.439,-0.001,-69.259,-127567091.348,75449343.128,7899.686,-0.797,1.303,45.468
-127297818.594,75201014.835,-60.933,23.439,-0.001,-69.260,-127567095.441,7544933

ConnectionAbortedError: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte